## 1.PP-YOLOv2 Introduction

As an important algorithm for object detection, the YOLO series adopts the one-stage method to greatly improve the detection speed, but the speed improvement also sacrifices some of the accuracy as a cost. Therefore, how to improve the accuracy of YOLOv3 while maintaining the speed of reasoning has become a key issue in its practical application.PP-YOLOv2 (R50) mAP in the COCO test dataset rises from 45.9% to 49.5%, an increase of 3.6 percentage points compared to v1. FP32 FPS is up to 68.9FPS, FP16 FPS is up to 106.5FPS, surpassing YOLOv4 and even YOLOv5! If RestNet101 is used as the backbone network, PP-YOLOv2 (R101) has up to 50.3% mAP and 15.9% faster than YOLOv5x with the same accuracy!

The PP-YOLO model is officially produced by PaddlePaddle and is a model of the YOLOv3 optimized and improved by PaddleDetection. More information about PaddleDetection can be found here https://github.com/PaddlePaddle/PaddleDetection.



## 2. Model Effects and Application Scenarios
### 2.1 Object Detection Tasks:

#### 2.1.1 Datasets:

The dataset is mainly in COCO format, which is divided into training set and test set.

#### 2.1.2 Model Effects:


The detection effect of PP-YOLOv2 on the picture is:

<div align="center">
<img src="https://user-images.githubusercontent.com/23690325/198869600-b7a549db-2cc6-49b1-8009-937fb5abe992.png"  width = "80%"  />
</div>

<div align="center">
<img src="https://user-images.githubusercontent.com/23690325/198869611-451eda5f-eda6-4717-902c-b9b06070bc72.png"  width = "80%"  />
</div>




## 3. How to Use the Model

### 3.1 Model Inference:
* Download 


In [ ]:
%cd /home/aistudio/work

!git clone https://gitee.com/paddlepaddle/PaddleDetection

* Installation

In [ ]:
# The script needs to be run in the PaddleDetection directory
%cd /home/aistudio/work/PaddleDetection/

# Install the required dependencies [already persisted, no need to install again].
!pip install pyzmq  -t /home/aistudio/external-libraries    
# After testing on AIstudio paddlepaddle 2.2.2, an error will occur, because pyzmq needs to be installed in advance.
!pip install -r requirements.txt

# The script needs to be run in the PaddleDetection directory.
%cd /home/aistudio/work/PaddleDetection/
# Set the python run directory.
%env PYTHONPATH=.:$PYTHONPATH
# Set GPU
%env CUDA_VISIBLE_DEVICES=0

!pip install pycocotools  
!pip install cython-bbox      
!pip install xmltodict  
!pip install terminaltables    
!pip intall motmetrics  
!pip install lap    
!pip install shapely      
!pip install pytest-benchmark    
!pip install pytest    


# Download PaddleDetection 
!python setup.py install  

* Verify whether the installation was successful or not.
If an error is reported, only perform the previous step.

In [ ]:
# Whether the installation was successful or not.
!python ppdet/modeling/tests/test_architectures.py

* Quick experience

Congratulations! Now that you've successfully installed PaddleDetection, let's get a quick feel at object detection.

In [ ]:
# Predict a picture on the GPU.
export CUDA_VISIBLE_DEVICES=0
python tools/infer.py -c configs/ppyolo/ppyolo_r50vd_dcn_1x_coco.yml -o use_gpu=true weights=https://paddledet.bj.bcebos.com/models/ppyolo_r50vd_dcn_1x_coco.pdparams --infer_img=demo/000000014439.jpg

An image with the predicted result is generated under the output folder.

The result is as follows:

![](https://ai-studio-static-online.cdn.bcebos.com/76fb0d0b60fe4fe39cc7302f4c25818133f970ebdf924d2d85f70f25a586aab9)


### 3.2 Model Training
* Clone the PaddleDetection repository (see 3.1 for details)

* Prepare the datasets.

In [ ]:
# return to /home/aistudio
%cd ~

# Review the extract directory
%cd /home/aistudio/work/PaddleDetection/
!tree -d dataset/wider_face

* Change yaml configurations files.



Change yaml configurations files``` configs/runtime.yml```

```
use_gpu: true 
log_iter: 20  
save_dir: output 
snapshot_epoch: 1 
print_flops: false

```
Change yaml configurations files``` configs/datasets/coco_detection.yml```

```
metric: COCO    
num_classes: 1  

TrainDataset:
  !COCODataSet
    image_dir: WIDER_train/images   
    anno_path: WIDERFaceTrainCOCO.json  
    dataset_dir: dataset/wider_face 
    data_fields: ['image', 'gt_bbox', 'gt_class', 'is_crowd']

EvalDataset:
  !COCODataSet
    image_dir: WIDER_val/images     
    anno_path: WIDERFaceValCOCO.json   
    dataset_dir: dataset/wider_face

TestDataset:
  !ImageFolder
    anno_path: WIDERFaceValCOCO.json
    
```
Change yaml configurations files``` configs/ppyolo/ppyolov2_r50vd_dcn_365e_coco.yml```

```
_BASE_: [
  '../datasets/coco_detection.yml',
  '../runtime.yml',
  './_base_/ppyolov2_r50vd_dcn.yml',
  './_base_/optimizer_365e.yml',
  './_base_/ppyolov2_reader.yml',
]

snapshot_epoch: 8   
weights: output/ppyolov2_r50vd_dcn_365e_coco/model_final
```



* Train the model.

In [ ]:
%cd /home/aistudio/work/PaddleDetection/
%env CUDA_VISIBLE_DEVICES=0
# Beginning training
!python  tools/train.py -c configs/ppyolo/ppyolov2_r50vd_dcn_365e_coco.yml  --use_vdl=true 

* Model evaluation

We provide ```configs/ppyolo/ppyolo_test.yml```for evaluating the effect of COCO test-dev2017 dataset, to evaluate the effect of COCO test-dev2017 dataset, you must first download the test-dev2017 dataset from the COCO dataset download page, and extract it to ```configs/ppyolo/ppyolo_test.yml```. The path configured in EvalReader.dataset and evaluated using the following command (attach the average accuracy AP and AR evaluation indicators of the model, and provide pictures or tables).

In [ ]:
%cd /home/aistudio/work/PaddleDetection/
%env CUDA_VISIBLE_DEVICES=0

!python tools/eval.py -c configs/ppyolo/ppyolov2_r50vd_dcn_365e_coco.yml  -o use_gpu=true

## 4. Model Principles
* Design Detection Net using Path Aggregation Network

PP-YOLOv2 uses one of FPN variations, PAN (Path Aggregation Network), to aggregate feature information from top to bottom.

![](https://ai-studio-static-online.cdn.bcebos.com/5f047e2e5f3c47efbb81c6cf3d81415e531133c1feff4f36a2cc13f88210ab69)




* Use the Mish activation function

PP-YOLOv2's mish activation function is applied to the detection neck instead of the skeleton network.

* Larger input size

Increasing the input size directly leads to an increase in the target area. This makes it easier for the network to capture information about small-sized targets for higher performance. However, larger inputs result in a larger memory footprint. So while using this strategy, PP-YOLOv2 also reduces the Batch Size.

## 5. Attention

Whether it is PP-YOLO or PP-YOLOv2, they are looking for the most cost-effective object detection solution in industrial practice, rather than simply stacking networks and strategies to improve the accuracy of single-stage object detection. The paper on PP-YOLOv2 also specifically mentioned that it is to show more network optimization methods for industry developers from the perspective of experimental reports, and these strategies can also be applied to the optimization of other networks, hoping to bring better networks to industry developers and more algorithm optimization inspiration. At the same time, when using the PP-YOLO series, attention should also be paid to:


* The PP-YOLO model uses train2017 from the COCO dataset as the training set, val2017 and test-dev2017 as the test set, and Box APtest evaluates the results for mAP (IoU=0.5:0.95).
* PP-YOLO model training process uses 8 GPUs, each GPU batch size is 24 for training, if the number of training GPUs and batch size do not use the above configuration, you must refer to the FAQ to adjust the learning rate and number of iterations.
* PP-YOLO model inference speed test is tested with single card V100, batch size=1, CUDA 10.2, CUDNN 7.5.1, and TensorRT inference speed test using TensorRT 5.1.2.2.
* The inference speed test data of PP-YOLO model FP32 is the inference speed benchmark test result using the Paddle prediction library using the --run_benchnark parameter in the deploy/python/infer .py script after exporting the model using the tools/export_model.py script, and the test is data that does not contain data preprocessing and model output post-processing (NMS) ( Consistent with YOLOv4 (AlexyAB) test method).
* Compared to FP32, the speed test of TensorRT FP16 removes the yolo_box (bbox decoding) part of the time-consuming, i.e. does not include data preprocessing, bbox decoding and NMS (consistent with YOLOv4 (AlexyAB) test method).



## 6. Related papers and citations
(If this model has relevant papers published, or is based on the results of certain papers, it can be here.)
References in Bibtex format are provided. ）

```
@article{huang2021pp,
  title={PP-YOLOv2: A Practical Object Detector},
  author={Huang, Xin and Wang, Xinxin and Lv, Wenyu and Bai, Xiaying and Long, Xiang and Deng, Kaipeng and Dang, Qingqing and Han, Shumin and Liu, Qiwen and Hu, Xiaoguang and others},
  journal={arXiv preprint arXiv:2104.10419},
  year={2021}
}
```

